In [8]:
import os

import pandas as pd
pd.set_option('display.max_columns', None)

import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
income_df = pd.read_csv(r'C:\Users\Garrett Pinkston\Desktop\TIM147\Project\data\kaggle_income.csv', encoding='iso-8859-1 ',)
income_df.head(2)

,id,State_Code,State_Name,State_ab,County,City,Place,Type,Primary,Zip_Code,Area_Code,ALand,AWater,Lat,Lon,Mean,Median,Stdev,sum_w
0,1011000,1,Alabama,AL,Mobile County,Chickasaw,Chickasaw city,City,place,36611,251,10894952,909156,30.771450,-88.079697,38773,30506,33101,1638.260513
1,1011010,1,Alabama,AL,Barbour County,Louisville,Clio city,City,place,36048,334,26070325,23254,31.708516,-85.611039,37725,19528,43789,258.017685


In [10]:
df = pd.read_csv(r'C:\Users\Garrett Pinkston\Desktop\TIM147\Project\data\RestaurantInfoComplete.csv')
df.head(3)

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,is_restaurant,checkinDate,starList,useful,funny,cool,StarSlope,population,pop_density,median_income,locations,chain
0,0,0,0,MTSW4McQd7CbVtyjqoe9mw,sthonorepastries,935 Race St,Philadelphia,PA,19107.0,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","['Restaurants', 'Food', 'Bubble Tea', 'Coffee ...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",True,"['2010-08-18 17:05:36', '2010-11-25 17:45:31',...","[4, 4, 3, 4, 3, 4, 5, 4, 4, 5, 5, 4, 4, 3, 3, ...","[6, 1, 1, 2, 2, 5, 1, 5, 0, 0, 0, 0, 0, 1, 0, ...","[0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, ...","[2, 1, 1, 1, 1, 3, 1, 4, 0, 0, 0, 0, 0, 0, 0, ...",0.831331,14984.0,10634.5,59706,1,not
1,1,1,1,CF33F8-E6oudUQ46HnavjQ,sonicdrivein,615 S Main St,Ashland City,TN,37015.0,36.269593,-87.058943,2.0,6,1,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","['Burgers', 'Fast Food', 'Sandwiches', 'Food',...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '...",True,"['2012-12-16 05:27:33', '2013-02-15 04:00:46',...","[4, 4, 1, 1, 2, 1]","[0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0]",-3.020524,18979.0,47.1,69514,48,medium
2,2,3,3,bBDDEgkFA1Otx9Lfe7BZUQ,sonicdrivein,2312 Dickerson Pike,Nashville,TN,37207.0,36.208102,-86.768170,1.5,10,1,"{'RestaurantsAttire': ""'casual'"", 'Restaurants...","['Ice Cream & Frozen Yogurt', 'Fast Food', 'Bu...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-21:0', '...",True,"['2011-01-17 15:31:21', '2011-02-23 21:28:08',...","[3, 2, 1, 1, 1, 1, 1, 1, 5, 1]","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",-0.477870,40697.0,812.3,300000,48,medium


In [25]:
import folium
from branca.colormap import LinearColormap

# filter to only florida
simple_df = df[df['state'] == 'FL']

# center around florida
us_map = folium.Map(location=[28.0400, -82.5090], zoom_start=10)

#title code HTML
title_html = """
             <h3 align="center" style="font-size:16px"><b>Interactive Map of Florida's Restaurants</b></h3>
             """
us_map.get_root().html.add_child(folium.Element(title_html))


#color map of restaurant reviews
colormap = LinearColormap(
    colors=['red', 'yellow', 'green'],
    vmin=simple_df['stars'].min(),
    vmax=simple_df['stars'].max()
)
colormap.add_to(us_map)


#iterate through, create groups for open and closed
#fill dependent on colormap
for grp_name, df_grp in simple_df.groupby('is_open'):
    feature_group = folium.FeatureGroup(name='Open' if (grp_name == 0) else 'Closed')
    for index, row in simple_df.iterrows():
        marker = folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=4,
            color=colormap(row['stars']),
            fill=True,
            fill_color=colormap(row['stars']),
            fill_opacity=0.7).add_to(feature_group)
    feature_group.add_to(us_map)

folium.LayerControl().add_to(us_map)

#create legend
legend_html = '''
    <div style="position: fixed; top: 10px; right: 10px; z-index:1000; font-size: 12px;">
      <b>Rating</b><br>
      <span style="display: inline-block; background-color: #FF0000; width: 15px; height: 15px;"></span> Low&nbsp;&nbsp;&nbsp;&nbsp;
      <span style="display: inline-block; background-color: #FFFF00; width: 15px; height: 15px;"></span> Medium&nbsp;&nbsp;&nbsp;&nbsp;
      <span style="display: inline-block; background-color: #008000; width: 15px; height: 15px;"></span> High
    </div>
'''
#apply legend to map
us_map.get_root().html.add_child(folium.Element(legend_html))


us_map.get_root().html.add_child(folium.Element(legend_html))

#save map
us_map.save('Florida_Interactive_Map.html')

# Shade not changing because points stacked on each other
# Screenshot Large US Map. Show interactive Florida map


In [27]:
import folium
from branca.colormap import LinearColormap

simple_df = income_df

# Centered in middle of USA, start zoom out
us_map = folium.Map(location=[28.0400, -82.5090], zoom_start=4)

#add title from up above
title_html = """
             <h3 align="center" style="font-size:16px"><b>InteractiveUSA Map with Median Income</b></h3>
             """
us_map.get_root().html.add_child(folium.Element(title_html))

#color map of median income
colormap = LinearColormap(
    colors=['red', 'yellow', 'green'],
    vmin=simple_df['Median'].min(),
    vmax=simple_df['Median'].max()
)

colormap.add_to(us_map)


for index, row in simple_df.iterrows():
    marker = folium.CircleMarker(
        location=[row['Lat'], row['Lon']],
        radius=4,
        color=colormap(row['Median']),
        fill=True,
        fill_color=colormap(row['Median']),
        fill_opacity=0.2).add_to(feature_group)
feature_group.add_to(us_map)
folium.LayerControl().add_to(us_map)

#save
us_map.save('us_map_with_income.html')
